# Obsah
- [O Guidance](../../../../code/01.Introduce)
- [Nastavenie](../../../../code/01.Introduce)
- [Nekontrolovaná generácia](../../../../code/01.Introduce)
- [Hovorenie za Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Výber](../../../../code/01.Introduce)
- [Reťazec myšlienok](../../../../code/01.Introduce)
- [Generovanie JSON](../../../../code/01.Introduce)
- [Generovanie HTML](../../../../code/01.Introduce)


# O Guidance
Guidance je osvedčená open-source knižnica pre Python, ktorá umožňuje kontrolovať výstupy akéhokoľvek jazykového modelu (LM). Jedným volaním API môžete v Pythone vyjadriť presné programové obmedzenia, ktoré musí model dodržiavať, a generovať štruktúrovaný výstup vo formáte JSON, Python, HTML, SQL alebo v akejkoľvek štruktúre, ktorú si daný prípad použitia vyžaduje.

Guidance sa líši od bežných techník vytvárania promptov. Uplatňuje obmedzenia tým, že model riadi token po tokene v inferenčnej vrstve, čím produkuje kvalitnejšie výstupy a znižuje náklady a latenciu až o 30–50 % pri použití v scenároch s vysokou štruktúrovanosťou.

Ak sa chcete dozvedieť viac o Guidance, navštívte [verejné úložisko na GitHube](https://github.com/guidance-ai/guidance) alebo si pozrite [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) na Microsoft Build.


# Nastavenie
1. Nainštalujte Guidance pomocou `pip install guidance --pre`
2. Nasadzujte Phi 3.5 mini endpoint v Azure tak, že prejdete na https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml a kliknete na tlačidlo "Deploy"
3. Uložte API kľúč vášho endpointu do environmentálnej premennej s názvom `AZURE_PHI3_KEY` a URL adresu do environmentálnej premennej s názvom `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Negenérované obmedzenia
Text môže byť generovaný bez akýchkoľvek obmedzení pomocou funkcie `gen()`. Toto je rovnaké ako používanie modelu bez Guidance.

## Formátovanie chatu
Podobne ako mnohé chatovacie modely, Phi-3 očakáva správy medzi používateľom a asistentom v špecifickom formáte. Guidance podporuje šablónu chatu Phi-3 a postará sa o formátovanie chatu za vás. Na vytvorenie jednotlivých častí konverzácie použite bloky `with user()` alebo `with assistant()`. Blok `with system()` môže byť použitý na nastavenie systémovej správy.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Úspora tokenov
V silne štruktúrovaných scenároch môže Guidance preskočiť tokeny a generovať iba potrebné tokeny, čím zlepšuje výkon, zvyšuje efektivitu a šetrí náklady na API. Generované tokeny sú v tomto zápisníku zobrazené so zvýrazneným pozadím. Vynútené tokeny sú zobrazené bez zvýraznenia a stoja rovnako ako vstupné tokeny, ktorých cena sa odhaduje na jednu tretinu ceny výstupných tokenov.

*Poznámka:* Prvý príklad s neobmedzenou generáciou nebol schopný vynútiť žiadne tokeny, pretože sme neposkytli žiadne obmedzenia.


# Hovorenie za Phi 3 
S pomocou Guidance môžete jednoducho vložiť text do odpovedí modelu. To môže byť užitočné, ak chcete nasmerovať výstup modelu určitým smerom.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Obmedzenie pomocou regexu
V predchádzajúcom príklade Phi 3 odpovedal s následnými vysvetleniami po zodpovedaní otázky slovom `Canberra`. Na obmedzenie výstupu modelu na presne jedno slovo je možné použiť regex.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# Výber z viacerých možností
Keď sú známe niektoré možné možnosti, môžete použiť funkciu `select()`, aby model vybral z ponúkaného zoznamu možností.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

S `select()` bol vygenerovaný iba token `Can`. Keďže `Canberra` je jedinou možnosťou, ktorá môže dokončiť odpoveď, zostávajúce tokeny boli vynútené.


# Reťaz myšlienok
Reťaz myšlienok je technika, ktorá môže pomôcť zlepšiť kvalitu výstupu modelu tým, že ho povzbudí k postupnému spracovaniu problému krok za krokom. Na dosiahnutie konečnej odpovede je zvyčajne potrebných viacero krokov v rámci výzvy. Najskôr modelu dajte pokyn, aby premýšľal krok za krokom. Potom model opäť vyzvite, aby poskytol konečnú odpoveď. Pri štandardných API na chatovú inferenciu to vyžaduje 2 volania API a modelom generovaná „reťaz myšlienok“ sa účtuje dvakrát – raz ako výstupné tokeny, keď ju model vygeneroval, a potom znova ako vstupné tokeny pri druhom volaní. S Guidance je celý viacstupňový proces spracovaný a účtovaný ako súčasť jedného volania API, čím sa znižujú náklady a latencia.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Generovanie JSON
Guidance môže byť použité na zabezpečenie generovania JSON, ktorý je v súlade s JSON schémou alebo pydantic modelom, ako je napríklad schéma používateľského profilu uvedená tu.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Generovanie HTML

Guidance môže byť použitý na generovanie kódu a dodržiavanie syntaktických požiadaviek v programovacom jazyku. V tejto sekcii vytvoríme malý Guidance program na písanie veľmi jednoduchých HTML webových stránok.

Webovú stránku rozdelíme na menšie sekcie, pričom každá bude mať svoju vlastnú Guidance funkciu. Tieto sekcie potom skombinujeme v našej finálnej funkcii na vytvorenie HTML webovej stránky. 
Následne spustíme túto funkciu na modeli s podporou Guidance v Azure AI.

*Poznámka:* Toto nebude plne funkčný generátor HTML; cieľom je ukázať, ako môžete vytvoriť štruktúrovaný výstup podľa vašich individuálnych potrieb.

Začíname importovaním toho, čo potrebujeme z Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML webové stránky sú veľmi štruktúrované a my budeme 'vynucovať' tieto časti stránky pomocou Guidance.  
Keď výslovne požadujeme text od modelu, musíme zabezpečiť, aby neobsahoval nič, čo by mohlo byť značkou - to znamená, že musíme vylúčiť znaky '<' a '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Potom môžeme použiť túto funkciu na generovanie textu v ľubovoľnom HTML tagu:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Teraz vytvorme hlavičku stránky.  
V rámci tohto kroku musíme vygenerovať názov stránky:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Telo HTML stránky bude naplnené nadpismi a odsekmi.  
Môžeme definovať funkciu na vykonanie každého:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Teraz funkcia na definovanie samotného tela HTML.  
Táto funkcia používa `select()` s `recurse=True` na generovanie viacerých nadpisov a odsekov:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Ďalej sa dostávame k funkcii, ktorá generuje kompletnú HTML stránku.  
Pridáme počiatočný HTML tag, potom vygenerujeme hlavičku, následne telo, a nakoniec pripojíme koncový HTML tag:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Poskytujeme používateľsky prívetivý obal, ktorý nám umožní:
- Nastaviť teplotu generovania
- Zachytiť vygenerovanú stránku z objektu Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Výstup môžeme potom zapísať do súboru:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

A [pozrite si výsledok](../../../../code/01.Introduce/sample_page.html).



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nenesieme zodpovednosť za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
